In [1]:
import pandas as pd
import os
import json
import numpy as np

In [2]:
def get_file_paths():
    '''
    Creates a list of all the file paths for the stocknet dataset
    '''
    directory = r"raw/"
    file_paths = []
    for folder in os.listdir(directory):
        try:
            for file in os.listdir(directory+ folder +"/"):
                new_path = directory+folder+"/"+file
                file_paths.insert(len(file_paths), new_path)  
        except:
            print("Could not open file for", folder)
        pass
    return file_paths

In [3]:
def format_txt_to_json_list(pathname):
    raw = open(pathname, "r")
    raw_text = raw.read()
    raw_text = raw_text[:-1]
    list_text = raw_text.split("\n")
    
    json_list = []
    for i in range(len(list_text)):
        new_json = json.loads(list_text[i])
        json_list.insert(i, new_json)
    
    return json_list

In [4]:
def get_tweets_from_json_list(json_list, ticker):
    
    temp = pd.DataFrame(columns={"date", "ticker", "message"})
    
    for val in json_list:
        message = val["text"]
        date = val["created_at"]
        ticker = ticker
        temp.loc[len(temp.index)] = [date, message, ticker]
    
    return temp

In [5]:
def format_files(paths):
    all_tweets = pd.DataFrame(columns={"ticker", "message", "date"})
    for path in paths:
        
        json_list = format_txt_to_json_list(path)
        ticker = path.split("/")[1]
        tweets = get_tweets_from_json_list(json_list, ticker)
        all_tweets = all_tweets.append(tweets)
        all_tweets.sort_values(by="date", inplace=True)
        all_tweets.reset_index(drop=True,inplace=True)
        
        
    return all_tweets

In [6]:
def main():
    
    paths = get_file_paths()
    
    tweets = format_files(paths)
    
    return tweets
    

In [7]:
stocknet_df = main()

Could not open file for .DS_Store


In [24]:
stocknet_df

,message,ticker,date
0,Tue Oct 14 14:29:22 +0000 2014,RT @NatureBiotech: News Analysis: Apple moves ...,AAPL
1,Mon Mar 30 10:11:17 +0000 2015,Hope I win this #iPad mini from @ValueWalk! $A...,AAPL
2,Mon May 05 10:25:21 +0000 2014,"Apple's iPad business isn't collapsing, but th...",AAPL
3,Fri Sep 19 14:20:59 +0000 2014,"HFTs rinsing $AAPL now, eyeing some calls for ...",AAPL
4,Fri Feb 28 18:48:05 +0000 2014,$AAPL can this drop 10% from here,AAPL
...,...,...,...
119839,Tue Aug 18 11:36:12 +0000 2015,Who's next after Samson Energy's bankruptcy $L...,XOM
119840,Wed Mar 04 06:31:03 +0000 2015,$XOM - Can AT&amp;T Save Your Retirement Portf...,XOM
119841,Sun Mar 06 16:27:34 +0000 2016,XLK Consumer Staples Select Sector SPDR Prev C...,XOM
119842,Sun Jan 25 18:30:10 +0000 2015,Reviewing Ensco's Recent Industry Presentation...,XOM


In [28]:
def export_as_csv(df, path):
    try:
        df.to_csv(path)
        print("Success")
    except:
        print("Unable to save to .csv")

In [53]:
# export_as_csv(stocknet_df, "stocknet_data.csv")

Success


In [18]:
len(stocknet_df["message"].unique())

95356

In [21]:
pd.to_datetime(stocknet_df["message"])

0        2014-10-14 14:29:22+00:00
1        2015-03-30 10:11:17+00:00
2        2014-05-05 10:25:21+00:00
3        2014-09-19 14:20:59+00:00
4        2014-02-28 18:48:05+00:00
                    ...           
119839   2015-08-18 11:36:12+00:00
119840   2015-03-04 06:31:03+00:00
119841   2016-03-06 16:27:34+00:00
119842   2015-01-25 18:30:10+00:00
119843   2015-06-18 19:37:38+00:00
Name: message, Length: 119844, dtype: datetime64[ns, UTC]

In [26]:
stocknet_df.rename(columns={"message":"date","ticker":"text","date":"ticker"}, inplace=True)

In [41]:
stocknet_df["date"] = pd.to_datetime(stocknet_df["date"])

In [45]:
export_as_csv(stocknet_df, "stocknet_data.csv")

Success


In [47]:
stocknet_df[stocknet_df["date"]=="C"]

,date,text,ticker


In [48]:
len(stocknet_df)

119844